***This notebooks assumes familiarity with the documentation on [how to add new games](howto_add_games.md) and how to [log events and build records](logging_and_scoring.md). Please read those first. Also first have a look at [how to prototype games](https://github.com/clp-research/clembench/blob/main/docs/howto_prototype_games.ipynb), which shows the basic commands to interact with the LLMs.***

# Making a new clemgame

## Index
[Game concept](#game-concept)

[Clemgame components](#clemgame-components)

[Clemgame template and workspace setup](#clemgame-template-and-workspace-setup)

[Instances and resources basics](#instances-basics-and-game-resources)

[Player basics]

[GameMaster implementation]

[GameBenchmark implementation]

[Testing and refining]

The core parts of a clemgame are separated into classes:
- **Player** class handles players of the game, holding their message history and making calls to a backend model to generate responses.
- **GameMaster** class handles the game loop, determining what each player gets prompted with when
- **GameScorer** class handles scoring based on episode records
- **GameBenchMark** class coordinates benchmark runs of the clemgame, initializing GameMaster with game instances and GameScorer with recorded episodes

# Game concept
The first step of creating a new clemgame is to come up with a game concept, or to find an existing dialogue game to adapt. In either case, it is important that the rules of the game can be implemented programmatically. While you can implement a clemgame with complex parsing, it is recommended to keep rules simple enough to understand benchmark results without learning code intricacies of the clemgame implementation.

## Example game concept
Let's implement a simple two-player game we will call `firstlast`.

The players should engage in a ~~turn-based~~ back-and-forth conversation about a predefined topic. Player A starts with an utterance whose first and last ~~token~~ word must start with a predefined letter, say  `d`. Player B must then reply with an utterance whose first and last ~~token~~ word must be the next one in the alphabet (here, an `e`). And so on, for `n` ~~turns~~ rounds (where each ~~turn~~ round comprises two turns, one with an utterance from A and one with an utterance from B). If an utterance does not conform to these rules (i.e. it is incorrect), the players lose the game. We also define a move format rule: If an utterance does not start with 'I SAY: ' (i.e., it is invalid), the game is immediately aborted. If all utterances up to turn ```n``` are valid and correct, the game is successful.

For instance, if the topic is `birds`, the initial letter is `h` and the number of ~~turns~~ rounds is 2, this would be a successful game:

- <ins>Round 1</ins>
- Player A: "Hi! I love birds, but it's hard to identify them. I need help." (h: hi / help)
- Player B: "I know what you mean. I can try to help, please describe it." (i: I/ it)
- <ins>Round 2</ins>
- Player A: "Just a moment... Ok, it's blue but looks like an Eurasian jay." (j: just / jay)
- Player B: "Kick in more details, otherwise I don't know." (k: kick / know)

At each turn, we need to check two aspects:
- Does the utterance have a valid form that can be parsed? Failing to meet this leads to the game being aborted. (Move format rule.)
- Does the utterance fulfils the game rules for a successful turn? Failing to meet this leads to game being lost. (Game rules.)

To make things simple, in this example we will check only these conditions:
- Validity: Does the utterance start with `I SAY:`?
- correctness: Do the first and last tokens begin with the same, correct letter at play?


# Clemgame components
A clemgame needs at least the following components:

1. [Game resources](#instances-basics-and-game-resources): All data, prompt templates and other files that are necessary to create instances of a game and to group these instances into experiments.
2. [Instances](#instances-basics-and-game-resources): A JSON file containing the configuration of each instance of this game, grouped into experiments. This must be done by a script named `instancegenerator.py`, with a class that inherits from `GameInstanceGenerator`.
3. [Players](#player-basics): A script that defines the programmatic behaviour and any other attributes of a player, inheriting from the `Player` base class. (This can be implemented in a file named `players.py`.)
4. [Game Master](#gamemaster-implementation): A script that controls and enforces the defined move and game rules and dynamics, inheriting from the `GameMaster` base class. This must be implemented in the `master.py` file.
5. [Game Benchmark](#gamebenchmark-implementation): A class that realises the game's running behavior for benchmarking, inheriting from `GameBenchmark`. This can also live in the file `master.py`.
6. [Game registry](#game-registry) entry: A JSON-format specification of the clemgame required by the `clemcore` framework to locate the clemgame's files.

Let's walk through the implementation step by step. We'll write the contents of `instancegenerator.py`, `master.py` and `players.py`, that you should save as files to run the game.

**Note**: For the mandatory methods, always check the parent class documentation to be sure about the required and optional arguments.

# Clemgame template and workspace setup
The most convenient way to create a clemgame is to start with the template repository. It contains a complete implementation of the `taboo` clemgame as a base to work from.

To start, follow these steps:
1. Clone the clemgame template repository from https://github.com/clembench/clemgame-template . The directory you clone it to will be the workspace directory.
2. Set up a python virtual environment with `clemcore`, as described in the [template readme](https://github.com/clembench/clemgame-template/blob/main/README.md).
3. Inspect the template's files and structure.
4. Create a directory for your clemgame, either copying the `taboo` directory and its contents, and renaming it for your game or by making a new directory. For the example game, create a `firstlast` directory.

Further implementation below is based on this template.

# Instances basics and game resources
Instances of the `taboo` clemgame are located under `clembench/taboo/in` in the `instances.json` file. All clemgames must follow this directory structure, with the clemgame's base directory containing a subdirectory named `in` which contains the `instances.json` file.

The `instances.json` file contains a JSON object with the single key `experiments`, which holds a list of JSON objects, each corresponding to an experiment. An experiment holds clemgame parameters that apply for all its instances, and is used to evaluate and compare scores between different variants of a clemgame. The keys `name`, to identify different experiments, and `game_instances` are mandatory for all experiment objects.

An experiment object's `game_instances` key holds a list of the experiment's instances, each containing different data that is needed by the Game Master (explained below) to play an individual episode of the clemgame.
## Game resources
Resources - located in the `resources` subdirectory - are files that are accessed for instance generation or by the Game Master running the clemgame.

Conventionally, initial prompts that explain the game and its rules, and contain the starting state for an episode, are stored as `.template` files in `resources/initial_prompts`. Clemcore provides convenient methods to load these `.template` files (see below).

As you can see in the `taboo` files, initial prompt templates usually contain placeholder strings like `$TARGET_WORD$` that are replaced with instance values to create individual initial prompts for each instance.

## Example clemgame instances
(In this example, defining an episode requires instantiating the initial prompts and 3 additional parameters: The topic, the letter for the first player and the number of turns for that game play.)

In this example, an instance defines which initial prompts to use and 3 additional parameters: The topic, the letter for the first player and the number of rounds that need to be successfully played.

### Defining prompts with game rules

The players need to be instructed on what the rules of the game are, possibly with some examples, at the beginning of the game. For that, we must define the initial prompts passed to player A and player B.

In the template text, we can define variables that will later be filled with our chosen values (here: topic, first letter, number of rounds). The prompts need to be adjusted for player A and B according to their roles. For example:

Player A:
> "Let's play a game. You must have a conversation about $TOPIC$ with your partner. Your first turn must start and end with words that begin with the letter $LETTER$. The reply of your partner must be similar, with the letter that comes after $LETTER$ in the alphabet. Then it's your turn again with the next letter, and so on. You'll do it for $N_TURNS$ turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."

Player B:
> "Let's play a game. You must have a conversation about $TOPIC$ with your partner. Their first turn must start and end with words that begin with the letter $LETTER$. Your reply must be similar, with the letter that comes after $LETTER$ in the alphabet. Then it's their turn again with the next letter, and so on. You'll do it for $N_TURNS$ turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."

You can later refine these initial prompts based on how models handle them. Save the initial prompts as plain texts using `.template` as an extension in the `initial_prompts` directory. You can save many templates if you wish to test different prompts, and read each of them when you generate game instances (see below). Note that this template is just an example and has not been tested. (Decide what amount of prompt engineering you will do at this step. Once you are satisfied with your preliminary results (i.e. the model can process the instructions well enough for your purposes), save to file...)

# Player basics

# GameMaster implementation

# GameBenchmark implementation

# Game registry